In [1]:
import pandas as pd
import logistic_regression as lr
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn import linear_model

In [2]:
e = pd.read_csv("./expression.short.csv", index_col=(0,1))
g = pd.read_csv("./g_train.csv", index_col=(0,1))

In [76]:
g_tissuespec = pd.read_csv("../../../../../data/annotations/annotation_train/tissue_spec_annotations_train_max.csv",index_col=(0,1))

In [77]:
g_tissuespec.head()

E027_general_promoter  E027_general_enhancer  \
subject_id gene_id                                                            
GTEX-N7MS  ENSG00000001561.6                       0                      0   
           ENSG00000003056.3                       0                      0   
           ENSG00000003402.15                      1                      0   
           ENSG00000004534.10                      0                      1   
           ENSG00000004660.10                      0                      0   

                               E027_TssA  E027_TssAFlnk  E027_EnhG  E027_Enh  \
subject_id gene_id                                                             
GTEX-N7MS  ENSG00000001561.6           0              0          0         0   
           ENSG00000003056.3           0              0          0         0   
           ENSG00000003402.15          1              0          0         0   
           ENSG00000004534.10          0              0          0         1   
           ENSG00000004660.10          0              0          0         0   

                               E027_TssBiv  E027_BivFlnk  E027_EnhBiv  \
subject_id gene_id                                                      
GTEX-N7MS  ENSG00000001561.6             0             0            0   
           ENSG00000003056.3             0             0            0   
           ENSG00000003402.15            0             0            0   
           ENSG00000004534.10            0             0            0   
           ENSG00000004660.10            0             0            0   

                               E062_general_promoter     ...       \
subject_id gene_id                                       ...        
GTEX-N7MS  ENSG00000001561.6                       0     ...        
           ENSG00000003056.3                       0     ...        
           ENSG00000003402.15                      1     ...        
           ENSG00000004534.10                      0     ...        
           ENSG00000004660.10                      0     ...        

                               E113_EnhBiv  E116_general_promoter  \
subject_id gene_id                                                  
GTEX-N7MS  ENSG00000001561.6             0                      0   
           ENSG00000003056.3             0                      0   
           ENSG00000003402.15            0                      1   
           ENSG00000004534.10            0                      1   
           ENSG00000004660.10            0                      0   

                               E116_general_enhancer  E116_TssA  \
subject_id gene_id                                                
GTEX-N7MS  ENSG00000001561.6                       0          0   
           ENSG00000003056.3                       0          0   
           ENSG00000003402.15                      0          1   
           ENSG00000004534.10                      0          1   
           ENSG00000004660.10                      1          0   

                               E116_TssAFlnk  E116_EnhG  E116_Enh  \
subject_id gene_id                                                  
GTEX-N7MS  ENSG00000001561.6               0          0         0   
           ENSG00000003056.3               0          0         0   
           ENSG00000003402.15              0          0         0   
           ENSG00000004534.10              1          0         0   
           ENSG00000004660.10              0          0         1   

                               E116_TssBiv  E116_BivFlnk  E116_EnhBiv  
subject_id gene_id                                                     
GTEX-N7MS  ENSG00000001561.6             0             0            0  
           ENSG00000003056.3             0             0            0  
           ENSG00000003402.15            0             0            0  
           ENSG00000004534.10            0             0            0  
           ENSG00000004660.10            0        

In [148]:
train = pd.concat([g, g_tissuespec], axis=1)

In [150]:
train.to_csv("../tissue_spec_tl/g_train_all.csv")

In [89]:
annotation_cols = list(g.columns)
annotation_cols.extend(list(g_tissuespec.columns))

In [81]:
annotation_cols

['max_GC_10kb',
 'max_CpG_10kb',
 'max_priPhCons_10kb',
 'max_mamPhCons_10kb',
 'max_verPhCons_10kb',
 'max_priPhyloP_10kb',
 'max_mamPhyloP_10kb',
 'max_verPhyloP_10kb',
 'max_GerpN_10kb',
 'max_GerpS_10kb',
 'max_dnaHelT_10kb',
 'max_dnaMGW_10kb',
 'max_dnaProT_10kb',
 'max_dnaRoll_10kb',
 'max_fitCons_10kb',
 'max_cHmmTssA_10kb',
 'max_cHmmTssAFlnk_10kb',
 'max_cHmmTxFlnk_10kb',
 'max_cHmmTx_10kb',
 'max_cHmmTxWk_10kb',
 'max_cHmmEnhG_10kb',
 'max_cHmmEnh_10kb',
 'max_cHmmZnfRpts_10kb',
 'max_cHmmHet_10kb',
 'max_cHmmTssBiv_10kb',
 'max_cHmmBivFlnk_10kb',
 'max_cHmmEnhBiv_10kb',
 'max_cHmmReprPC_10kb',
 'max_cHmmReprPCWk_10kb',
 'max_cHmmQuies_10kb',
 'max_EncH3K27Ac_10kb',
 'max_EncH3K4Me1_10kb',
 'max_EncH3K4Me3_10kb',
 'max_EncNucleo_10kb',
 'max_EncOCCombPVal_10kb',
 'max_EncOCDNasePVal_10kb',
 'max_EncOCFairePVal_10kb',
 'max_EncOCpolIIPVal_10kb',
 'max_EncOCctcfPVal_10kb',
 'max_EncOCmycPVal_10kb',
 'max_EncOCDNaseSig_10kb',
 'max_EncOCFaireSig_10kb',
 'max_EncOCpolIISig_10kb'

In [90]:
train["labels"] = sklearn.preprocessing.binarize(np.abs(train["median"]).reshape(-1,1), threshold=1.5)

In [82]:
t = ['Heart_Atrial_Appendage','Heart_Left_Ventricle', 'Muscle_Skeletal', 'Esophagus_Muscularis']
train = train.dropna(subset=t, thresh=3)
train["median_muscles"] = np.abs(train[t]).median(axis=1)
train["labels_muscles"] = sklearn.preprocessing.binarize(np.abs(train["median_muscles"]).reshape(-1,1), threshold=1.5)

In [92]:
G = train[annotation_cols]
G.insert(0,"intercept",1)

In [93]:
E = train["labels"]

In [94]:
fpr, tpr, auc = dict(), dict(), dict()

In [95]:
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(G, E, test_size=0.33, random_state=42)

In [108]:
G.shape

(158674, 364)

In [101]:
lambdas = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6]

In [109]:
beta

array([ -6.79034582e+00,   2.57497669e-01,   2.05317497e-01,
         1.77442945e-01,   3.06439701e-01,   5.24348740e-01,
        -7.51817153e-01,  -1.91316305e+00,   3.72557947e+00,
        -1.55608043e-01,  -8.44419271e-02,  -5.53267049e-02,
         1.15015703e-01,  -9.13447528e-02,   4.56873792e-02,
         4.27876416e-02,  -2.82968763e-01,   3.85044467e-02,
        -2.03066680e+00,   1.65371244e-01,  -4.08548877e-02,
         5.91342755e-01,   4.65186210e-02,   6.88288074e-01,
        -1.43724615e-01,   1.14640706e+00,  -2.66976627e+00,
        -1.29237456e+00,   6.57865152e-02,  -1.42094767e-01,
         1.89098211e-01,  -9.09427914e-01,  -8.25691724e-01,
        -1.07702296e+00,   6.43161793e-01,   6.05862466e-02,
        -1.79999682e-02,  -5.26858185e-01,   9.96315672e-02,
        -1.01326952e-01,   2.87022003e-01,  -2.23939951e-01,
         1.14097274e+00,  -5.78967003e-01,   3.87387039e-01,
        -8.50465845e-01,  -7.82915692e-01,   2.05880634e+00,
         2.77735256e-01,

In [110]:
fpr, tpr, auc, betas = dict(), dict(), dict(), dict()
#for i in range(len(lambdas)):
lambda_hp = 1.0
beta_small = lr.sgd(x_train, y_train, beta, np.zeros(G.shape[1]), lambda_hp)
betas[lambda_hp] = beta_small
fpr[lambda_hp], tpr[lambda_hp], _ = sklearn.metrics.roc_curve(y_test, np.exp(lr.log_prob(x_test, beta_small)))
auc[lambda_hp] = sklearn.metrics.auc(fpr[lambda_hp], tpr[lambda_hp])
print(auc[lambda_hp])

KeyError: 1.0

In [114]:
fpr[lambda_hp], tpr[lambda_hp], _ = sklearn.metrics.roc_curve(y_test, np.exp(lr.log_prob(x_test, beta_small)))
auc[lambda_hp] = sklearn.metrics.auc(fpr[lambda_hp], tpr[lambda_hp])
print(auc[lambda_hp])

0.571323139802


In [119]:
beta_shared = beta_small

In [140]:
#t = ['Heart_Atrial_Appendage','Heart_Left_Ventricle', 'Muscle_Skeletal', 'Esophagus_Muscularis']
t = ['Heart_Left_Ventricle']
train_t = pd.concat([train[annotation_cols], train[t]], axis=1)
train_t = train_t.dropna()
#train = train[train[t].notnull()]
#train = train.dropna(subset=t, thresh=3)
#train["median_heart"] = np.abs(train[t]).median(axis=1)
train_t["labels_heart"] = sklearn.preprocessing.binarize(np.abs(train_t[t]), threshold=1.5)
G = train_t[annotation_cols]
G.insert(0,"intercept",1)
E = train_t["labels_heart"]
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(G, E, test_size=0.33, random_state=42)

In [141]:
len(x_train)

51175

In [124]:
lambda_hp = 1.0
beta = lr.sgd(x_train, y_train, np.ones(G.shape[1]), beta_shared, lambda_hp)
betas[lambda_hp] = beta
fpr[lambda_hp], tpr[lambda_hp], _ = sklearn.metrics.roc_curve(y_test, np.exp(lr.log_prob(x_test, beta)))
auc[lambda_hp] = sklearn.metrics.auc(fpr[lambda_hp], tpr[lambda_hp])
print(auc[lambda_hp])

0.529599245149


In [144]:
lambda_hp = 1e6
beta = lr.sgd(x_train, y_train, np.ones(G.shape[1]), beta_shared, lambda_hp)
betas[lambda_hp] = beta
fpr[lambda_hp], tpr[lambda_hp], _ = sklearn.metrics.roc_curve(y_test, np.exp(lr.log_prob(x_test, beta)))
auc[lambda_hp] = sklearn.metrics.auc(fpr[lambda_hp], tpr[lambda_hp])
print(auc[lambda_hp])

0.509510200719


In [117]:
np.savetxt("../tissue_spec_tl/shared_beta.csv", beta_small, newline=' ')

In [87]:
beta_l2 = lr.sgd(x_train, y_train, np.ones(G.shape[1]), np.zeros(G.shape[1]), .000001)
beta_sharedprior = lr.sgd(x_train, y_train, np.ones(G.shape[1]), shared_beta, .000001)

KeyboardInterrupt: 

In [ ]:
beta_sharedprior_001 = lr.sgd(x_train, y_train, np.ones(G.shape[1]), shared_beta, .001)
beta_sharedprior_1 = lr.sgd(x_train, y_train, np.ones(G.shape[1]), shared_beta, 1)
beta_sharedprior_1000 = lr.sgd(x_train, y_train, np.ones(G.shape[1]), shared_beta, 1000)

In [72]:
fpr['shared_beta_000001'], tpr['shared_beta_000001'], _ = sklearn.metrics.roc_curve(y_test, np.exp(lr.log_prob(x_test, beta)))
auc['shared_beta_000001'] = sklearn.metrics.auc(fpr['shared_beta_000001'], tpr['shared_beta_000001'])

In [73]:
auc

{0: 0.52273512351842288,
 1: 0.46840111047731248,
 2: 0.52213398541797273,
 3: 0.52214381269377319,
 'shared_beta_001': 0.52657103683922979,
 'shared_beta_10000': 0.54012668401896657,
 'shared_beta_000001': 0.52213357291503781,
 'shared_beta_10000000': 0.5391744816560915,
 'shared_beta_100': 0.5369354885204316}

In [30]:
shared_beta = beta

In [118]:
shared_beta

array([ -6.96294145e+00,   1.98099493e-01,   3.19441631e-01,
         1.89189895e-01,   2.57645342e-01,   5.12669708e-01,
        -7.24370891e-01,  -1.91166630e+00,   3.69574484e+00,
        -1.51593842e-01,  -7.39906071e-02,  -5.79078900e-02,
         1.47863872e-01,  -7.70098066e-02,   2.59872459e-02,
         4.13219458e-02,   1.72410267e-01,   4.58247997e-01,
        -2.10161718e+00,   1.79301252e-01,  -1.21863916e-03,
         4.75591058e-01,   5.62524309e-02,   7.14131189e-01,
        -8.12227693e-02,   4.71016876e-01,  -1.00838145e+00,
        -8.93944720e-01,   2.06819646e-01,  -1.25128892e-01,
         2.46272879e-01,  -1.14868111e+00,  -8.57281549e-01,
        -6.16125122e-01,   5.45720713e-01,   1.33243243e-01,
        -2.81991012e-02,  -6.60804803e-01,   2.32806512e-02,
        -8.54830981e-02,   2.85078669e-01,  -3.81406172e-01,
         1.36112313e+00,  -2.62543115e-01,   3.15371121e-01,
        -7.81566144e-01,  -1.00192982e+00,   2.20316905e+00,
         3.61176496e-01,